# Replica project

### The data

In [2]:
# loading the metadata
%load_ext autoreload
%autoreload 2

import pandas as pd

from torch import nn
import torchvision.models as models
from tqdm import tqdm
import torch
import pickle
from glob import glob
from scipy import sparse


from utils import *

path = '/home/guhennec/scratch/2021_Cini/TopologicalAnalysis_Cini/data/'
data_dir = '/scratch/students/schaerf/'
replica_dir = '/mnt/project_replica/datasets/cini/'



In [2]:
metadata = pd.read_csv(path + 'Cini_20210811.csv', sep=';')
metadata.head(5)

/tmp/ipykernel_2741047/2286784503.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(path + 'Cini_20210811.csv', sep=';')


,Drawer,ImageNumber,Author,Description,AuthorOriginal,AuthorULANLabel,AuthorULAN,AuthorURL,AuthorComplemented,AuthorComplement,...,AuthorDeath,AuthorBirthLong,AuthorBirthLat,AuthorDeathLong,AuthorDeathLat,AuthorBirthCity,AuthorDeathCity,uid,CardboardURL,ImageURL
0,1A,1,ALLEGRINI Francesco,tca con penna acquarellata 148 x 203 mm,"ALLEGRINI, Francesco","Allegrini, Francesco",ulan:500115272,http://vocab.getty.edu/page/ulan/500115272,0,No complement,...,1690.0,NaN,NaN,NaN,NaN,NaN,NaN,253993c139284a45be233a13121ddeeb,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_1....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...
1,1A,2,BAROCCI Federico,Studio di figura penna acquarellata (carboncin...,"BAROCCI, Federico","Barocci, Federico",ulan:500115210,http://vocab.getty.edu/page/ulan/500115210,0,No complement,...,1612.0,12.633333,43.716667,12.633333,43.716667,Urbino,Urbino,1323356994c24635a11fdcd9d5f9284a,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_2....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...
2,1A,3,BASSANO Leandro,Scene penna su carta blu 171 x 219 mm,"BASSANO, Leandro","Bassano, Leandro",ulan:500015945,http://vocab.getty.edu/page/ulan/500015945,0,No complement,...,1622.0,11.733333,45.766667,12.326667,45.438611,Bassano del Grappa,Venice,a4268385f6384e61a3dd092bc6b8c083,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_3....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...
3,1A,4,CAMPIGLI Massimo,Ragazza su una scala gessetto su carta bruna g...,"CAMPIGLI, Massimo","Campigli, Massimo",ulan:500029770,http://vocab.getty.edu/page/ulan/500029770,0,No complement,...,1971.0,13.400000,52.516667,6.639811,43.269316,Berlin,Var,550f368cdb4442aab4d5e2ada702d6ad,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_4....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...
4,1A,5,CARRACCI A attr,Madonna e Santi penna 291 x 205 mm,"CARRACCI, A. attr.",NaN,NaN,NaN,1,attr,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6047de5547b643cb87491be925748bee,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_5....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...


In [3]:
metadata['uid'].nunique(), metadata.shape

(329480, (330002, 37))

In [4]:
# dropping duplicates

metadata = metadata.groupby('uid').first().reset_index()

In [5]:
# in jeanne's code
with open(path + 'save_link_data_2018_08_02.pkl', 'rb') as f:
    morpho_graph_complete = pickle.load(f)

# contains uid	img1	img2	type	annotated
morpho_graph_complete.head()

,uid,img1,img2,type,annotated
0,0f84770be893475581bebd8c0ce95704,4b6e5a18012e4974a43f6f652c9cad78,e3329a91a266412fb1a0975d70894178,POSITIVE,2017-07-19 15:02:50.997512+00:00
1,5ade49fc2dc649b9947496a41e2decdb,384b9f017d5f43ba9dcfd92f2af830c4,e3329a91a266412fb1a0975d70894178,POSITIVE,2017-07-19 15:02:51.238615+00:00
2,bf4d99eaf7da4bd486f9007c90c177a5,94fe74bcebf9442fb5cd9a0327590a84,726f47fb5bd94c5db8069334cc58020e,POSITIVE,2017-07-20 12:18:23.705228+00:00
3,b29ec3c83c5641f9b6afadb2409b6746,922fe92d61e744b1a9462c848a916bcf,726f47fb5bd94c5db8069334cc58020e,POSITIVE,2017-07-20 12:18:24.021703+00:00
4,ab27b55997b94bef9cca122829e9f64d,94fe74bcebf9442fb5cd9a0327590a84,922fe92d61e744b1a9462c848a916bcf,POSITIVE,2017-07-20 12:18:23.873097+00:00


In [6]:
morpho_graph_complete['type'].value_counts() # does positive mean similar?

DUPLICATE        114283
POSITIVE           6293
NON-DUPLICATE        60
PROPOSAL             27
Name: type, dtype: int64

In [7]:
morpho_graph_complete['uid'].nunique(), morpho_graph_complete.shape

(120663, (120663, 5))

In [8]:
positives = get_train_test_split(metadata, morpho_graph_complete)
positives.set.value_counts()

train    2877
test     1099
val       579
Name: set, dtype: int64

In [9]:
positives = positives.groupby('uid_connection').first().reset_index()
print(positives.shape)
positives.head()

(2862, 45)


,uid_connection,index,uid,Drawer,ImageNumber,Author,Description,AuthorOriginal,AuthorULANLabel,AuthorULAN,...,AuthorBirthCity,AuthorDeathCity,CardboardURL,ImageURL,img1,img2,type,annotated,cluster,set
0,002ca9c06a704c779e69d93cb4fadaa3,649,45588b4e33444ed8b34c523bac808bc4,158B,556,TIZIANO Vecellio (copia da),S.Mariá Maddalena. cm. 123 x 91,TIZIANO (copia da -),Titian,ulan:500031075,...,Belluno,Venice,https://dhlabsrv4.epfl.ch/iiif_cini/158B%2F158...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,45588b4e33444ed8b34c523bac808bc4,da54c106180449d1a52993e9a5bc17fd,POSITIVE,2018-07-24 14:58:44.559049+00:00,50,train
1,0038ab462149452b98a4d7359a06f9f4,1571,ae828b07e4284d80b652206ffb77278a,158C,120,TIZIANO Vecellio (copia da),"Morte di S. Pietro Martire. mm. 123,8 x 84,2",TIZIANO (copia da -),Titian,ulan:500031075,...,Belluno,Venice,https://dhlabsrv4.epfl.ch/iiif_cini/158C%2F158...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,ae828b07e4284d80b652206ffb77278a,ea64207684694d379579998f88b7c29a,POSITIVE,2018-07-16 09:47:57.987515+00:00,51,test
2,00474897de044f7fb1ef16888dc198f3,1107,76b2c4aeab404bc79768117e3ddc62b1,47A,858,BELLINI Giovanni (cerchia di),Pietà.,"BELLINI, Giovanni (cerchia di -)","Bellini, Giovanni",ulan:500019244,...,Venice,Venice,https://dhlabsrv4.epfl.ch/iiif_cini/47A%2F47A_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,76b2c4aeab404bc79768117e3ddc62b1,b8dd2c0c2c7146e68e5bbb0aaf6cf803,POSITIVE,2018-03-14 10:46:45.603341+00:00,498,val
3,009c03c265b8418495da038bc3826bf7,1764,c5839e5220e5419e92d878396b0c04bb,135C,57,SEC XVI ex,Un soffitto. Part.: Leda e il cigno.,SEC. XVI ex.,None,None,...,None,None,https://dhlabsrv4.epfl.ch/iiif_cini/135C%2F135...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,450ee2aceb014709b23a93e1163937c5,c5839e5220e5419e92d878396b0c04bb,POSITIVE,2018-05-07 08:28:11.797723+00:00,92,train
4,00aeddb93a8a4120aa984eda6c822747,747,4d02860f58f2443ba4d1ef088756f854,21C,228,BASSANO Leandro,"La preparazione del pranzo. cm. 88,3 x 102,9","BASSANO, Leandro.","Bassano, Leandro",ulan:500015945,...,Bassano del Grappa,Venice,https://dhlabsrv4.epfl.ch/iiif_cini/21C%2F21C_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,4d02860f58f2443ba4d1ef088756f854,27bad0f213a342c3987e8cec61e40550,POSITIVE,2018-03-23 15:41:53.094226+00:00,218,train


## Getting a subset of the data

In order to select a subset of the data, we would like to vary spatially and temporally, for it to be suitable to the data in the morphograph and to be comprehensive of what we want to study.


In [10]:
#subset_max = metadata[metadata['BeginDate'] > 1550][metadata['BeginDate'] < 1650][metadata['AuthorDeathCity'].isin(['Parma', 'Bologna', 'Ravenna', 'Verona', 'Ferrara'])].reset_index()

subset_max = metadata[metadata['BeginDate'] > 1450][metadata['BeginDate'] < 1650]#[metadata['AuthorDeathCity'].isin(['Parma', 'Bologna', 'Ravenna', 'Verona', 'Ferrara'])].reset_index()
subset_max.shape

/tmp/ipykernel_2741047/1639676227.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subset_max = metadata[metadata['BeginDate'] > 1450][metadata['BeginDate'] < 1650]#[metadata['AuthorDeathCity'].isin(['Parma', 'Bologna', 'Ravenna', 'Verona', 'Ferrara'])].reset_index()


(100589, 37)

In [11]:
subset_max = remove_duplicates(subset_max, morpho_graph_complete)
subset_max.shape

(80210, 44)

In [12]:
list_downloaded = ['/'.join(file.split('/')[-2:]) for file in glob('/mnt/project_replica/datasets/cini/*/*')]
list_downloaded[0]

'74B/74B_22.jpg'

In [13]:
len(list_downloaded)

330682

In [14]:
del morpho_graph_complete
del metadata

In [15]:
#uniques = subset_max[subset_max['uid'].isin(list_downloaded)].groupby('uid').first().reset_index()
subset_max['uid2'] = subset_max['Drawer'].astype(str) + '/' + subset_max['Drawer'].astype(str) + '_' + subset_max['ImageNumber'].astype(str) + '.jpg'
uniques = subset_max[subset_max['uid2'].isin(list_downloaded)].groupby('uid').first().reset_index()
print(uniques.shape)

(80080, 45)


In [16]:
#uniques['Desc'] = uniques['Description'].str.lower().str.split()
#words = uniques.explode('Desc').reset_index()['Desc'].value_counts()
#print(words[words.index.str.len() > 4][words > 100].to_string())

In [17]:
uniques = uniques[uniques['Description'].notnull()]

In [18]:
short = uniques[uniques['Description'].str.split().str.len() == 1]
#print(short['Description'])
short['Desc'] = short['Description'].str.lower().str.strip('.').str.split()
words = short.explode('Desc').reset_index()['Desc'].value_counts()
for idx in words[words.index.str.len() > 4][words > 10].index:
    #print(idx)
    continue

/tmp/ipykernel_2741047/1446786371.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short['Desc'] = short['Description'].str.lower().str.strip('.').str.split()


In [19]:
to_remove = ['affreschi', 'tavolo', 'affresco', 'veduta', 'disegno', 'composizione', 'stucchi', 'studi', ]
short_remove = short.explode('Desc').reset_index()
short_remove = short_remove[short_remove['Desc'].isin(to_remove)]
uniques = uniques[~uniques['uid'].isin(list(short_remove['uid']))]
uniques.shape

(79564, 45)

In [20]:
uniques['uid_connection'] = uniques['uid_connection'].fillna('Not uid connection') 

/tmp/ipykernel_2741047/3999118373.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniques['uid_connection'] = uniques['uid_connection'].fillna('Not uid connection')


In [21]:
data = pd.concat([uniques, positives], axis=0).reset_index().groupby(['uid', 'uid_connection']).last().reset_index()
data_sample = pd.concat([uniques.sample(10000), positives], axis=0).reset_index()#.groupby('uid').first().reset_index()
print(data.shape)
data = data[data['Drawer'].notnull()]
data_sample = data_sample[data_sample['Drawer'].notnull()]
print(data.shape)
data['path'] = data['Drawer'].astype(str) + '/' + data['Drawer'].astype(str) + '_' + data['ImageNumber'].astype(str) + '.jpg'
data_sample['path'] = data_sample['Drawer'].astype(str) + '/' + data_sample['Drawer'].astype(str) + '_' + data_sample['ImageNumber'].astype(str) + '.jpg'

(82426, 47)
(82426, 47)


In [22]:
uid2path = {}

for uid, path in tqdm(zip(data['uid'].unique(), data['path'].unique())):
    uid2path[uid] = path

with open(data_dir + 'uid2path.pkl', 'wb') as outfile:
    pickle.dump(uid2path, outfile)

80208it [00:00, 1248980.84it/s]


In [23]:
data['uid'].nunique(), data['path'].nunique()

(80208, 80208)

In [24]:
data.sample(5)

,uid,uid_connection,level_0,cluster,index,Drawer,ImageNumber,Author,Description,AuthorOriginal,...,AuthorDeathCity,CardboardURL,ImageURL,img1,img2,type,annotated,uid2,set,path
49055,8cd1a2fe81ab45578bc90ee05386b0e3,4561257233f945df991e642dc8af5b4c,47673,23097,64421,19B,491,PALLADIO Andrea,Facciata di una chiesa. Penna 405 x 275.,"PALLADIO, Andrea.",...,Vicenza,https://dhlabsrv4.epfl.ch/iiif_cini/19B%2F19B_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,8cd1a2fe81ab45578bc90ee05386b0e3,c41a954da93c4ea69f3fca7642e681d0,DUPLICATE,2018-07-19 17:45:40.717914+00:00,19B/19B_491.jpg,None,19B/19B_491.jpg
5755,0e732574df8f4376b94094fa2eb4973c,Not uid connection,5615,0e732574df8f4376b94094fa2eb4973c,6679,91C,178,VECELLIO Marco (?),Crono con altre figure allegoriche.,"VECELLIO, Marco (?)",...,Venice,https://dhlabsrv4.epfl.ch/iiif_cini/91C%2F91C_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,None,None,None,NaT,91C/91C_178.jpg,None,91C/91C_178.jpg
78740,f241f86ed21c447f883b43378383eb31,Not uid connection,76482,f241f86ed21c447f883b43378383eb31,110831,31C,346,MARMI Giacinto,"""Progetti di restauri per i palazzi e le ville...","MARMI, Giacinto.",...,None,https://dhlabsrv4.epfl.ch/iiif_cini/31C%2F31C_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,None,None,None,NaT,31C/31C_346.jpg,None,31C/31C_346.jpg
5981,0efab3f65a204c51afaf5b193d41165b,3118a60d33424a59b3033e241282d3ca,5835,26424,6959,63C,397,VITTORIA Alessandro,Monumento A. Contarini,"VITTORIA, Alessandro.",...,None,https://dhlabsrv4.epfl.ch/iiif_cini/63C%2F63C_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,0efab3f65a204c51afaf5b193d41165b,918f1e0e7d7044b8a1e92e592c6b6b0d,DUPLICATE,2018-07-19 18:06:01.157131+00:00,63C/63C_397.jpg,None,63C/63C_397.jpg
9396,17e5ef291f2b4686a4657c83cd91e537,Not uid connection,9164,17e5ef291f2b4686a4657c83cd91e537,11094,24A,90,DA CARAVAGGIO Polidoro,"Composizione religiosa. Pennello, colore a cor...",POLIDORO da CARAVAGGIO,...,Messina,https://dhlabsrv4.epfl.ch/iiif_cini/24A%2F24A_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,None,None,None,NaT,24A/24A_90.jpg,None,24A/24A_90.jpg


In [25]:
#data.to_csv(data_dir + 'full_data.csv')
data_sample.to_csv(data_dir + 'dedup_data_sample.csv')
data.to_csv(data_dir + 'dedup_data.csv')


In [ ]:
data = pd.read_csv(data_dir + 'dedup_data.csv')
print(data['set'].value_counts())
data[data['set'].isin(['train', 'test', 'val'])].shape, data_sample[data_sample['set'].isin(['train', 'test', 'val'])].shape

In [4]:
data_old = pd.read_csv(data_dir + 'full_data.csv')
data_old[data_old['set'].isin(['train', 'test', 'val'])].shape

(2862, 47)

### Creating embeddings of all the images

In [28]:
model = models.resnet50(pretrained=True)
#print(model)

In [29]:
newmodel = torch.nn.Sequential(
    *(list(model.children())[:-2]), nn.AdaptiveMaxPool2d((1,1))
)
#print(newmodel)


In [30]:
img = preprocess_image(data_dir + 'subset/' + data['uid'][0] + '.jpg')
embedding = get_embedding(img, newmodel).squeeze(3).squeeze(1).squeeze(0)

In [31]:
embeddings = [get_embedding(preprocess_image(data_dir + 'subset/' + uid + '.jpg'), newmodel).squeeze(3).squeeze(1).squeeze(0) for uid in tqdm(data['uid'].unique())]

100%|██████████| 4357/4357 [16:40<00:00,  4.35it/s]


In [32]:
embeddings_new = get_lower_dimension(embeddings)

In [33]:
embeddings[0].shape, embeddings_new[0].shape 

((2048,), (100,))

### Finding most similar embeddings

In [34]:
tree = make_tree_list(embeddings_new)

In [60]:
train_test = data[data['set'].notnull()].drop(columns=['level_0']).reset_index()

Cs = []
for i in tqdm(range(train_test.shape[0])):
    list_theo = list(train_test[train_test['img1'] == train_test['uid'][i]]['img2']) + list(train_test[train_test['img2'] == train_test['uid'][i]]['img1']) + [train_test['uid'][i]]
    list_ = find_most_similar_list(train_test['uid'][i], tree, embeddings_new, list(data['uid'].unique()), list_theo)
    Cs.append(list_)

len(Cs)


100%|██████████| 2862/2862 [00:10<00:00, 286.12it/s]


2862

In [58]:
print(train_test[train_test['img1'].isin(list_downloaded)].shape)
print(train_test[train_test['img2'].isin(list_downloaded)].shape)

(2299, 47)
(2256, 47)


In [61]:
train_test['C'] = Cs

train_test = train_test[train_test['img1'].isin(list_downloaded)]
train_test = train_test[train_test['img2'].isin(list_downloaded)]

final = train_test[['img1', 'img2', 'C', 'set']].explode('C')
final.columns = ['A', 'B', 'C', 'set']
print(final.shape)
final.sample(5)

(31709, 4)


,A,B,C,set
1771,6fcf1af179fa47488d40564e80856a1e,e100fa4b1d1e4abab65256ac310ab543,6bf773f2e04f4c919ccd6f0b76df2b15,train
1539,ef25d7d6549547b59945a61ae6849d3d,272cd39493464a878264554be785e9db,92fc36e4d04743ec97e441342acd29f1,test
63,59fb95590dda4ff58e63dc0df43e70c3,a043a8ccb7994ef68179ad66929143af,756a2003514740448ffb37eec9351177,train
289,383cd750b1cc425089ff3cf3d2c81df1,8b47beb1f1a6438993f3988e880cfa9c,45d519d9ba884c948b2a0abf1a692599,test
581,3d7b289477ca4765a6932bd85cf51fec,246ca4d9a3324ad2aad8ec6a4e1c6769,4ced1a6287c1417db8b41e79e1a04c79,test


In [62]:
final[final['C'].isin(list_downloaded)].shape

(31709, 4)

In [63]:
final[final['set'] == 'train'].reset_index().to_csv(data_dir + 'abc_train.csv')
final[final['set'] == 'test'].reset_index().to_csv(data_dir + 'abc_test.csv')
